<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/01_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [20]:
pip install duckdb 

In [21]:
import pandas as pd

In [22]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

## Create the the tables
DuckDB

In [23]:
con.execute("CREATE SCHEMA IF NOT EXISTS uni;")

In [24]:
con.execute("CREATE TABLE S (\
    Matr VARCHAR(45),\
    SNome VARCHAR(45),\
    Citta VARCHAR(45),\
    ACorso INT,\
    UNIQUE (Matr),\
    PRIMARY KEY (Matr)\
);")

con.execute("CREATE TABLE D(\
 CD VARCHAR(45),\
 CNome VARCHAR(45),\
 Citta VARCHAR(45),\
 UNIQUE (CD),\
 PRIMARY KEY (CD)\
);")

con.execute("CREATE TABLE C(\
 CC VARCHAR(45),\
 CNome VARCHAR(45),\
 CD VARCHAR(45),\
 UNIQUE (CC),\
 PRIMARY KEY (CC)\
);")

con.execute("CREATE TABLE E(\
 Matr VARCHAR(45),\
 CC VARCHAR(45),\
 DATA DATE,\
 VOTO INT,\
 UNIQUE (Matr, CC),\
 PRIMARY KEY (Matr, CC)\
);")


con.execute("INSERT INTO S (Matr, SNome, Citta, ACorso)\
VALUES\
('M1','Lucia Quaranta','SA',1),\
('M2','Giacomo Tedesco','PA',2),\
('M3','Carla Longo','MO',1),\
('M4','Ugo Rossi','MO',1),\
('M5','Valeria Neri','MO',2),\
('M6','Giuseppe Verdi','BO',1),\
('M7','Maria Rossi',null,1);")


con.execute("INSERT INTO D (CD, CNome, Citta)\
VALUES\
('D1','Paolo Rossi','MO'),\
('D2','Maria Pastore','BO'),\
('D3','Paola Caboni','FI');")



con.execute("INSERT INTO C (CC,CNome, CD)\
VALUES\
('C1','Fisica 1','D1'),\
('C2','Analisi Matematica 1','D2'),\
('C3','Fisica 2','D1'),\
('C4','Analisi Matematica 2','D3');")



con.execute("INSERT INTO E (Matr,CC,Data,Voto)\
VALUES\
('M1','C1','1995-06-29',24),\
('M1','C2','1996-08-09',33),\
('M1','C3','1996-03-12',30),\
('M2','C1','1995-06-29',28),\
('M2','C2','1996-07-07',24),\
('M3','C2','1996-07-07',27),\
('M3','C3','1996-11-11',25),\
('M4','C3','1996-11-11',33),\
('M6','C2','1996-01-02',28),\
('M7','C1','1995-06-29',24),\
('M7','C2','1996-04-11',26),\
('M7','C3','1996-06-23',27);")


## Query the DB

In [49]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

con.execute(q) # execute the query
con.fetchdf() # return a pandas dataframe

,matr,snome,citta,acorso,cc,cnome,cd,data,voto
0,M1,Lucia Quaranta,MO,1,C1,Paolo Rossi,D1,1995-06-29,24
1,M1,Lucia Quaranta,MO,1,C2,Paolo Rossi,D1,1996-08-09,33
2,M1,Lucia Quaranta,MO,1,C3,Paolo Rossi,D1,1996-03-12,30
3,M2,Lucia Quaranta,MO,1,C1,Paolo Rossi,D1,1995-06-29,28
4,M2,Lucia Quaranta,MO,1,C2,Paolo Rossi,D1,1996-07-07,24
...,...,...,...,...,...,...,...,...,...
1003,M4,Maria Rossi,FI,1,C3,Paola Caboni,D3,1996-11-11,33
1004,M6,Maria Rossi,FI,1,C2,Paola Caboni,D3,1996-01-02,28
1005,M7,Maria Rossi,FI,1,C1,Paola Caboni,D3,1995-06-29,24
1006,M7,Maria Rossi,FI,1,C2,Paola Caboni,D3,1996-04-11,26


In [60]:

# Operatori relazionali : <attr> <op-rel> <cost> 
# dove <op-rel> ∈ {=, <>, >, >=, <, <=}
# 
# Studenti del secondo anno di corso
q = ''' 
SELECT *
FROM S
WHERE ACorso=2;
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso
0,M2,Giacomo Tedesco,PA,2
1,M5,Valeria Neri,MO,2


In [65]:
# Esami con voto compreso tra 24 e 28
q = '''
SELECT *
FROM E
WHERE Voto > 24
AND Voto <= 28;
'''
con.execute(q) 
con.fetchdf()


,matr,cc,data,voto
0,M2,C1,1995-06-29,28
1,M3,C2,1996-07-07,27
2,M3,C3,1996-11-11,25
3,M6,C2,1996-01-02,28
4,M7,C2,1996-04-11,26
5,M7,C3,1996-06-23,27


In [64]:
# Operatore di set : <attr> IN (<cost1>, ..., <costN>)
#
# Esami con voto pari a 29, 30 oppure con lode (voto pari a 33)
q = '''
SELECT *
FROM E
WHERE Voto IN (29,30,33);
'''
con.execute(q) 
con.fetchdf()


,matr,cc,data,voto
0,M1,C2,1996-08-09,33
1,M1,C3,1996-03-12,30
2,M4,C3,1996-11-11,33


In [57]:
# Operatore di confronto stringhe : <attr> LIKE <stringa> 
# dove <stringa> puo` contenere i caratteri speciali   (carattere arbitrario) e % (stringa arbitraria)
#
# Studenti il cui nome inizia con A e termina con O

q = '''
SELECT *
FROM S
WHERE SNome LIKE 'V%i'
'''
con.execute(q) 
con.fetchdf()


,matr,snome,citta,acorso
0,M2,Giacomo Tedesco,PA,2
1,M5,Valeria Neri,MO,2


In [66]:
# Ordinamento del risultato: ORDER BY <attr> [ASC|DESC]
#
# Studenti di Modena ordinati in senso ascendente rispetto all’anno di corso
q = '''
SELECT Matr,ACorso 
FROM S
WHERE Citta = 'MO' 
ORDER BY ACorso DESC
'''
con.execute(q) 
con.fetchdf()


,matr,acorso
0,M5,2
1,M3,1
2,M4,1


In [67]:
# L’ordinamento deve essere fatto rispetto a uno o piu' elementi della <lista-select>: 
# un tale elemento puo' essere indicato anche riportando la sua posizione nella <lista-select>.
#
# Esami del corso C1 ordinati in senso discendente rispetto al voto espresso in sessantesimi, 
# e a parita` di voto rispetto alla matricola

q = '''
SELECT Matr,CC,(60*Voto)/30 
FROM E
WHERE CC='C1'
ORDER BY 3 DESC, Matr DESC
'''

con.execute(q) 
con.fetchdf()


,matr,cc,60 * voto / 30
0,M2,C1,56
1,M7,C1,48
2,M1,C1,48


## JOIN operators

In [68]:
# Patriamo dal prodotto cartesiano: 
# Cioe' la combinazione di tutte le possibili tuple delle tabelle elencate
# nella clausola FROM
q = '''
SELECT *
FROM S,D,E,C
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso,cd,cnome,cc,data,voto
0,M1,Lucia Quaranta,MO,1,D1,Fisica 1,C1,1995-06-29,24
1,M1,Lucia Quaranta,MO,1,D1,Fisica 1,C1,1996-08-09,33
2,M1,Lucia Quaranta,MO,1,D1,Fisica 1,C1,1996-03-12,30
3,M2,Lucia Quaranta,MO,1,D1,Fisica 1,C1,1995-06-29,28
4,M2,Lucia Quaranta,MO,1,D1,Fisica 1,C1,1996-07-07,24
...,...,...,...,...,...,...,...,...,...
1003,M4,Maria Rossi,FI,1,D3,Analisi Matematica 2,C4,1996-11-11,33
1004,M6,Maria Rossi,FI,1,D3,Analisi Matematica 2,C4,1996-01-02,28
1005,M7,Maria Rossi,FI,1,D3,Analisi Matematica 2,C4,1995-06-29,24
1006,M7,Maria Rossi,FI,1,D3,Analisi Matematica 2,C4,1996-04-11,26


In [69]:
q = '''
SELECT * 
FROM S,E
'''

con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso,cc,data,voto
0,M1,Lucia Quaranta,SA,1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,C3,1996-03-12,30
3,M2,Lucia Quaranta,SA,1,C1,1995-06-29,28
4,M2,Lucia Quaranta,SA,1,C2,1996-07-07,24
...,...,...,...,...,...,...,...
79,M4,Maria Rossi,NaN,1,C3,1996-11-11,33
80,M6,Maria Rossi,NaN,1,C2,1996-01-02,28
81,M7,Maria Rossi,NaN,1,C1,1995-06-29,24
82,M7,Maria Rossi,NaN,1,C2,1996-04-11,26


In [70]:
# Il Join, per definizione e' un "filtro" sul prodotto cartesiano
# cioe', in algebra relazionale, una selezione sul prodotto cartesiano
q = '''
SELECT *
FROM S,E
WHERE S.Matr=E.Matr
'''
con.execute(q) 
con.fetchdf()

# che equivale a:
# q = '''
# SELECT *
# FROM S JOIN E on S.Matr=E.Matr
# '''
# con.execute(q) 
# con.fetchdf()

# q = '''
# SELECT *
# FROM S natural JOIN E
# '''
# con.execute(q) 
# con.fetchdf()

,matr,snome,citta,acorso,cc,data,voto
0,M1,Lucia Quaranta,SA,1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,C3,1996-03-12,30
3,M2,Giacomo Tedesco,PA,2,C1,1995-06-29,28
4,M2,Giacomo Tedesco,PA,2,C2,1996-07-07,24
5,M3,Carla Longo,MO,1,C2,1996-07-07,27
6,M3,Carla Longo,MO,1,C3,1996-11-11,25
7,M4,Ugo Rossi,MO,1,C3,1996-11-11,33
8,M6,Giuseppe Verdi,BO,1,C2,1996-01-02,28
9,M7,Maria Rossi,NaN,1,C1,1995-06-29,24


In [71]:
# Pero' possono esseci studenti che non hanno sostenuto esami.
# Ad esempio:

q = '''
SELECT *
FROM S
WHERE S.Matr not in (SELECT Matr FROM E)
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso
0,M5,Valeria Neri,MO,2


In [72]:
# Se volessimo anche questi studenti nell'output del join
# dobbiamo usare un LEFT join

q = '''
SELECT * FROM S LEFT JOIN E on S.Matr=E.Matr;
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso,cc,data,voto
0,M1,Lucia Quaranta,SA,1,C3,1996-03-12,30.0
1,M2,Giacomo Tedesco,PA,2,C2,1996-07-07,24.0
2,M3,Carla Longo,MO,1,C3,1996-11-11,25.0
3,M4,Ugo Rossi,MO,1,C3,1996-11-11,33.0
4,M6,Giuseppe Verdi,BO,1,C2,1996-01-02,28.0
5,M7,Maria Rossi,NaN,1,C3,1996-06-23,27.0
6,M1,Lucia Quaranta,SA,1,C2,1996-08-09,33.0
7,M2,Giacomo Tedesco,PA,2,C1,1995-06-29,28.0
8,M3,Carla Longo,MO,1,C2,1996-07-07,27.0
9,M7,Maria Rossi,NaN,1,C2,1996-04-11,26.0


In [73]:
# E se volessi:
# Combinazioni di studenti e di docenti residenti nella stessa citta'
# inclusi gli studenti che risiedono in una citta' 
# che non ha corrispondenza nella relazione dei docenti
q = '''
SELECT *
FROM S JOIN D ON S.Citta=D.Citta;
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso,cd,cnome
0,M3,Carla Longo,MO,1,D1,Paolo Rossi
1,M4,Ugo Rossi,MO,1,D1,Paolo Rossi
2,M5,Valeria Neri,MO,2,D1,Paolo Rossi
3,M6,Giuseppe Verdi,BO,1,D2,Maria Pastore


In [74]:
# con anche gli studenti di citta' da cui non viene nessun docente:
q = '''
SELECT *
FROM S LEFT JOIN D ON S.Citta=D.Citta;
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso,cd,cnome
0,M3,Carla Longo,MO,1,D1,Paolo Rossi
1,M4,Ugo Rossi,MO,1,D1,Paolo Rossi
2,M5,Valeria Neri,MO,2,D1,Paolo Rossi
3,M6,Giuseppe Verdi,BO,1,D2,Maria Pastore
4,M1,Lucia Quaranta,NaN,1,NaN,NaN
5,M2,Giacomo Tedesco,NaN,2,NaN,NaN
6,M7,Maria Rossi,NaN,1,NaN,NaN


In [75]:
# Nella clausola FROM e' possibile esprimere anche piu' condizioni di join.
# 
# Ad esempio, se volessimo:
# Per ogni esame con voto superiore a 24 riportare il nome dello studente 
# e il codice del docente del corso
q = '''
SELECT S.SNome,C.CD, E.voto
FROM (S JOIN E ON (S.Matr=E.Matr))
		JOIN C ON (E.CC=C.CC)
WHERE Voto > 24;
'''
con.execute(q) 
con.fetchdf()

,snome,cd,voto
0,Lucia Quaranta,D2,33
1,Lucia Quaranta,D1,30
2,Giacomo Tedesco,D1,28
3,Carla Longo,D2,27
4,Carla Longo,D1,25
5,Ugo Rossi,D1,33
6,Giuseppe Verdi,D2,28
7,Maria Rossi,D2,26
8,Maria Rossi,D1,27


In [76]:
# che equivale a:
q = '''
SELECT S.SNome,C.CD, E.voto
FROM S,E,C
WHERE S.Matr=E.Matr 
AND E.CC=C.CC
AND Voto > 24;
'''
con.execute(q) 
con.fetchdf()

,snome,cd,voto
0,Lucia Quaranta,D2,33
1,Lucia Quaranta,D1,30
2,Giacomo Tedesco,D1,28
3,Carla Longo,D2,27
4,Carla Longo,D1,25
5,Ugo Rossi,D1,33
6,Giuseppe Verdi,D2,28
7,Maria Rossi,D2,26
8,Maria Rossi,D1,27


In [77]:
# Con gli outer join, certi attributi vengono duplicati (per definizione)
# Ad sempio:
q = '''
SELECT S.Matr,S.Citta,D.CD, D.Citta
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

,matr,citta,cd
0,M3,MO,D1
1,M4,MO,D1
2,M5,MO,D1
3,M6,BO,D2
4,M1,NaN,NaN
5,M2,NaN,NaN
6,M7,NaN,NaN


In [78]:
# come faccio a fare un merge e considerare solo una Citta'?
# funzione: COALESCE(...) sceglie il primo valore non nullo
q = '''
SELECT S.Matr, S.Citta, D.Citta,D.CD
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

,matr,citta,cd
0,M3,MO,D1
1,M4,MO,D1
2,M5,MO,D1
3,M6,BO,D2
4,M1,NaN,NaN
5,M2,NaN,NaN
6,M7,NaN,NaN


In [79]:
q = '''
SELECT S.Matr, COALESCE(S.Citta, D.Citta) AS Citta, D.CD AS CodiceDocente
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

,matr,citta,codicedocente
0,M3,MO,D1
1,M4,MO,D1
2,M5,MO,D1
3,M6,BO,D2
4,M1,SA,NaN
5,M2,PA,NaN
6,M7,NaN,NaN


### Self-Join

In [80]:
# Quando il join avviene tra la tabella e se stessa
# 
# Ad esempio: 
# Selezionare le coppie di studenti della stessa citta'

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta;
'''

con.execute(q) 
con.fetchdf()

,snome,citta
0,Lucia Quaranta,SA
1,Giacomo Tedesco,PA
2,Valeria Neri,MO
3,Valeria Neri,MO
4,Valeria Neri,MO
5,Giuseppe Verdi,BO
6,Ugo Rossi,MO
7,Ugo Rossi,MO
8,Ugo Rossi,MO
9,Carla Longo,MO


In [81]:
# Cosi' pero' selezionaimo anche uno studente e se stesso
# Quindi modifichiamo aggiungendo:

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta
AND S1.Matr < S2.Matr;
'''

con.execute(q) 
con.fetchdf()

#
# DOMANDA:
# perche' <> non andrebbe bene?
#

,snome,citta
0,Valeria Neri,MO
1,Valeria Neri,MO
2,Ugo Rossi,MO


In [82]:
# Matricole degli studenti che hanno sostenuto almeno 
# uno degli esami sostenuti dallo studente di nome 'Giuseppe Verdi'

q = '''
SELECT E1.Matr
FROM S, E E1, E E2
WHERE E2.Matr = S.Matr
AND E1.CC = E2.CC
AND S.SNome='Giuseppe Verdi';
'''

con.execute(q) 
con.fetchdf()

,matr
0,M1
1,M2
2,M3
3,M6
4,M7


In [84]:
q = '''
SELECT E1.Matr
FROM E E1
WHERE E1.CC IN (SELECT E2.CC
				FROM E E2, S
				WHERE S.SNome='Giuseppe Verdi'
				AND S.Matr = E2.Matr);
'''

con.execute(q) 
con.fetchdf()

,matr
0,M1
1,M2
2,M3
3,M6
4,M7


## Nested Quereis

In [ ]:
# Interrogazioni innestate

# Una interrogazione viene detta innestata o nidificata se la sua condizione
# e' formulata usando il risultato di un’altra interrogazione, 
# chiamata subquery

q = '''
SELECT *
FROM S
WHERE S.Matr IN (SELECT Matr
				 FROM E);
'''

con.execute(q) 
con.fetchdf()

## ANY/IN/EXISTS
- Operatori quantificati: `<attr><op-rel>[ANY|ALL]<subquery`
- Operatore di set: `<attr> [NOT] IN <subquery>`
- Quantificatore esistenziale : `[NOT] EXISTS <subquery>`

In [85]:
#  TIPO [ANY/ALL]
# Per esprimere una condizione che e' vera quando 
# qualsiasi (ANY) opppure tutti (ALL) gli elementi
# restituiti dalla subquery fanno verificare la condizione
# i.e., condizione risulta TRUE
 
# ANY
# Nome degli studenti che hanno sostenuto l’esame del corso C1

q = '''
SELECT SNome
FROM S
WHERE Matr = ANY ( SELECT Matr
                        FROM   E
                        WHERE  CC='C1');
'''
con.execute(q) 
con.fetchdf()


,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [86]:
q = '''
SELECT SNome
FROM S natural JOIN E
WHERE E.CC = 'C1';
'''
con.execute(q) 
con.fetchdf()

,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [88]:
# ALL
# Studenti con anno di corso piu` basso
q = '''
SELECT *
FROM S
WHERE ACorso <= ALL ( SELECT ACorso
					  FROM S);
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso
0,M1,Lucia Quaranta,SA,1
1,M3,Carla Longo,MO,1
2,M4,Ugo Rossi,MO,1
3,M6,Giuseppe Verdi,BO,1
4,M7,Maria Rossi,NaN,1


In [89]:
# Spesso possono esserci diversi modi di esprimere la stesa query
# es: Nome degli studenti che hanno sostenuto l’esame del corso C1
# senza subquery:
q = '''
SELECT S.SNome
FROM E join S on E.Matr=S.Matr
WHERE E.CC = 'C1';
'''
con.execute(q) 
con.fetchdf()

# # oppure:
# q = '''
# SELECT S.SNome
# FROM S
# WHERE S.Matr in (SELECT E.Matr 
# 				 FROM E WHERE E.CC='C1');
# '''
# con.execute(q) 
# con.fetchdf()

,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [90]:
# ALTRO ESEMPIO:
# Nome degli studenti che hanno sostenuto l’esame di un corso del docente D1
#/
q = '''
SELECT SNome
FROM E,S,C
WHERE E.Matr=S.Matr 
AND E.CC=C.CC
AND CD='D1';
'''
con.execute(q) 
con.fetchdf()

,snome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Ugo Rossi
5,Maria Rossi
6,Maria Rossi


In [91]:
# che puo' essere espressa anche come:
q = '''
SELECT SNome
FROM S
WHERE S.Matr IN (SELECT E.Matr 
				 FROM E 
				 WHERE E.CC IN (SELECT C.CC 
				 				FROM C 
				 				WHERE C.CD='D1'));
'''
con.execute(q) 
con.fetchdf()

,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Carla Longo
3,Ugo Rossi
4,Maria Rossi


In [92]:
# Provare la precedente query partendo dalla subquery piu' interna

# Fino ad ora abbiamo considerato delle subquery dove
# la query innestata era indipendente dalla query esterna

# Se vogliamo pero' selezionare:
# Per ogni citta', il nome degli studenti con anno di corso piu` alto
q = '''
SELECT S1.Citta, S1.SNome, S1.ACorso
FROM S S1
WHERE S1.ACorso >= ALL (SELECT S2.ACorso
					  	FROM S S2
					  	WHERE S1.Citta=S2.Citta);
'''
con.execute(q) 
con.fetchdf()
# S1 e' la tupla corrente

,citta,snome,acorso
0,SA,Lucia Quaranta,1
1,PA,Giacomo Tedesco,2
2,MO,Valeria Neri,2
3,BO,Giuseppe Verdi,1
4,NaN,Maria Rossi,1


### EXISTS

In [94]:
# QUALIFICATORE ESISTENZIALE:

# EXISTS

# Mettendo in relazione subquery e query estenza con una clausola
# che è vera solo se per la tupla corrente il risultato della subquery
# restituisce qualcosa (i.e., non è nullo).
# Da qui: "EXISTS", cioè ESISTE qualcosa che rende TRUE quel predicato.

# Ad esempio:
# Nome degli studenti che hanno sostenuto l’esame del corso C1:

q = '''
SELECT SNome
FROM S
WHERE EXISTS (SELECT *
			  FROM E
			  WHERE E.Matr=S.Matr
			  AND E.CC = 'C1');
'''
con.execute(q) 
con.fetchdf()			  

,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [95]:
# Allo stesso modo, il negato della clausola EXISTS:

# NOT EXISTS (<subquery>)

# ha valore true se e solo se l’insieme di valori restituiti da <subquery> e' vuoto.

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1
q = '''
SELECT SNome
FROM S
WHERE NOT EXISTS ( SELECT *
				   FROM E
				   WHERE E.Matr=S.Matr 
				   AND E.CC='C1' )
'''
con.execute(q) 
con.fetchdf()

# Tip:
# Se state usando EXISTS senza una subquery crrelata,
# molto probabilmente state commettendo un errore:

# Perche'?
# (hint: cosa restituisce la subquery?) 

,snome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


### Riduzione di query innestate

In [96]:
# Le query innestate formulate con i seguenti operatori 
# si possono ridurre a query di join equivalenti 
# (stessa risposta per ogni possibile istanza della base di dati):
# • IN
# • ANY (con qualsiasi operatore di confronto) 
# • EXISTS con subquery correlata
# 

				   
# Nome degli studenti che hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr IN ( SELECT Matr
				FROM E
				WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 2. 
q = '''
SELECT SNome 
FROM S
WHERE Matr =ANY ( SELECT Matr
				  FROM E
				  WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 3.
q = '''
SELECT SNome 
FROM S
WHERE EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
con.execute(q) 
con.fetchdf()

			   
# QUERY EQUIVALENTE CON JOIN
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC='C1'
'''
con.execute(q) 
con.fetchdf()

,snome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [97]:
# INVECE:
 
# Le query innestate formulate con i seguenti operatori 
# non si possono ridurre:

# • NOT IN
# • ALL (con qualsiasi operatore di confronto) 
# • NOT EXISTS con subquery correlata

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr NOT IN ( SELECT Matr
	 				FROM E
					WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 2.
q = '''
SELECT SNome 
FROM S
WHERE Matr <>ALL ( SELECT Matr
				  FROM E
				  WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 3.
q = '''
SELECT SNome 
FROM S
WHERE NOT EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
con.execute(q) 
con.fetchdf()

# NON SI POSSONO ESPRIMRE CON JOIN

,snome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


In [98]:
# NOTARE: la differenza con la seguente query
# ERRATA (per la consegna):
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC <> 'C1'
'''
con.execute(q) 
con.fetchdf()
# cosa restituisce?

,snome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Carla Longo
5,Ugo Rossi
6,Giuseppe Verdi
7,Maria Rossi
8,Maria Rossi


## Funzioni di Aggregazione

In [99]:
# 		Funzioni aggregate 
# 		(column functions)

# Dato un insieme di tuple che soddisfano un predicato
# (e.g., il risultato di una query qualsiasi di quelle
# viste finora)
# le funzione di aggregazione restituiscono un risultato
# "aggregato" calcolato su queell'insieme.

# Possono essere cosi' facilmente ritrovati:
# La media dei voti di uno studete;
# Il voto massimo in un dato esame;
# Il numero di studenti che proviene da una data citta;
# ... 

q = '''
SELECT COUNT(*)
FROM S
'''
con.execute(q) 
con.fetchdf()

,count_star()
0,7


In [100]:
q = '''
SELECT AVG(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

,avg(e.voto)
0,27.416667


In [101]:
q = '''
SELECT MAX(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()


,max(e.voto)
0,33


In [102]:
q = '''
SELECT MIN(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

,min(e.voto)
0,24


In [103]:
q = '''
SELECT SUM(E.VOTO)/COUNT(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

,sum(e.voto) / count(e.voto)
0,27.0


In [104]:
q = '''
SELECT DISTINCT(E.CC)
FROM E
'''
con.execute(q) 
con.fetchdf()

,cc
0,C1
1,C2
2,C3


In [105]:
q = '''
SELECT E.CC
FROM E
'''
con.execute(q) 
con.fetchdf()

,cc
0,C1
1,C2
2,C3
3,C1
4,C2
5,C2
6,C3
7,C3
8,C2
9,C1


In [107]:
q = '''
SELECT COUNT(DISTINCT(E.CC))
FROM E
'''
# Alcuni DBMS non permettono questa sintassi
con.execute(q) 
con.fetchdf()

,count(e.cc)
0,3


In [108]:
# Es. “Numero di studenti presenti” 
q = '''
SELECT COUNT(Matr)
FROM S
'''
con.execute(q) 
con.fetchdf()

,count(matr)
0,7


In [109]:
# Es. “Numero di studenti che hanno sostenuto almeno un esame” 
q = '''
SELECT COUNT(DISTINCT Matr)
FROM E
'''
con.execute(q) 
con.fetchdf()

,count(matr)
0,6


In [110]:
# Es. “Numero di studenti con anno di corso non nullo” 
q = '''
SELECT COUNT(ACorso)
FROM S
'''
con.execute(q) 
con.fetchdf()

,count(acorso)
0,7


In [112]:
# Es. “Numero di anni di corso di studenti presenti” 
q = '''
SELECT COUNT(DISTINCT ACorso)
FROM S
'''
con.execute(q) 
con.fetchdf()

,count(acorso)
0,2


In [115]:
# Es. “Numero di coppie distinte matricola-voto” 
q = '''
SELECT COUNT(DISTINCT Matr, Voto)
FROM E
'''
# Alcuni DBML non permettono questa sintassi
con.execute(q) 
con.fetchdf()

RuntimeError: ignored

In [118]:
# Es. “Voto medio degli esami sostenuti dalla matricola M1”
q = '''
SELECT AVG(Voto)
FROM E 
WHERE Matr='M1'
'''
con.execute(q) 
con.fetchdf()


# che e' equivalente a
q = '''
SELECT SUM(Voto)/COUNT(Voto) 
FROM E 
WHERE Matr='M1'
'''
con.execute(q) 
con.fetchdf()

,sum(voto) / count(voto)
0,29.0


In [119]:
# Es. “Studenti il cui anno di corso `e minore di quello massimo presente”
q = '''
SELECT *
FROM S
WHERE ACorso < ( SELECT MAX(ACorso)
				 FROM S)
'''
con.execute(q) 
con.fetchdf()

,matr,snome,citta,acorso
0,M7,Maria Rossi,NaN,1
1,M3,Carla Longo,MO,1
2,M1,Lucia Quaranta,SA,1
3,M4,Ugo Rossi,MO,1
4,M6,Giuseppe Verdi,BO,1


In [120]:
# Considerate la sequente query, con funzione di aggregazione:

q = '''
SELECT Matr,MAX(Voto)
FROM E
'''
con.execute(q) 
con.fetchdf()
# PERCHE' E' SBAGLIATA?



RuntimeError: ignored